# Flask Playground

### This notebook containers Selenium Simulation Code that would preform the data capture from ecWIN.

In [1]:
import sys
import json
import datetime
import time

import pprint
pp = pprint.PrettyPrinter(indent=4)

from pymongo import MongoClient
from bson.objectid import ObjectId
from influxdb import InfluxDBClient
from influxdb import DataFrameClient


import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

import time
import datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import get_close_matches
import random

## Configure Mongodb

In [2]:
mongoClient = MongoClient('localhost',27017)
db = mongoClient.witsECOMP

## Configure InfluxDB

In [3]:
databaseName = "timeseriesdata"
influxClient = DataFrameClient("localhost", 8086, "root", "root", database=databaseName)
influxClient.create_database(databaseName)

In [4]:
# influxClient.drop_database(databaseName)

## Read in time Series from excel spreadsheet
Note: this process can take a long time (2 to 3 minutes!).

In [5]:
spreadsheet = pd.read_excel('/home/chris/Desktop/Wits Data/WITS ecWIN7 Data/WITS ecWIN7 Data 2018.xlsx',
                            sheet_name='WITS ecWIN7 Data 2018',
                            index_col="DateTime"
                           )

## Read in building names and geojson information
These files were manually created by generating geojson from http://geojson.io/ and identifying the corisponding building names.

In [6]:
buildingSensorNames = spreadsheet.columns
print(buildingSensorNames)

Index(['WITS 13 Jubilee Road_kVA', 'WITS 13 Jubilee Road_kVarh',
       'WITS 13 Jubilee Road_kWh', 'WITS 3 Jubilee Road_kVA',
       'WITS 3 Jubilee Road_kVarh', 'WITS 3 Jubilee Road_kWh',
       'WITS 5 Jubilee Road_kVA', 'WITS 5 Jubilee Road_kVarh',
       'WITS 5 Jubilee Road_kWh', 'WITS 9 Jubilee Road_kVA',
       ...
       'WITS WC Village Zesti Lemonz_kWh', 'WITS WC WITS Club GEN 100 kVa_kVA',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kVarh',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kWh', 'WITS WC Wits Club_kVA',
       'WITS WC Wits Club_kVarh', 'WITS WC Wits Club_kWh',
       'WITS Wits Club Summation_kVA', 'WITS Wits Club Summation_kVarh',
       'WITS Wits Club Summation_kWh'],
      dtype='object', length=986)


In [7]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'DayStyle_Maximum': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'WeekStyle_Maximum': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'YearStyle_Maximum': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'DayStyle_Average': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'WeekStyle_Average': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'YearStyle_Average': {'color': '#999', 'fillColor': '#999', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}, 'buildingName': 'Chamber of Mines', 'buildingId': 0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28

In [8]:
#Itterate over all buildings defined in the geojson and for each find the most likely sensor from the excel spreadhseet
#this is effectivly preforming a join between the two sets, based off weak links (the names)
for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    #Find the most likly sensor for that particular building
    mostLiklySensor = get_close_matches(buildingName + "_kVA", buildingSensorNames)[0]
    dataFrame = spreadsheet[mostLiklySensor].to_frame()
    influxClient.write_points(dataFrame, databaseName,{'buildingNumber': index, 'buildingName': buildingName, 'sensorName': mostLiklySensor})
    print(buildingName + "->" + mostLiklySensor)

Chamber of Mines->WITS WC Chamber of Mines Total_kVA
Flower Hall->WITS WC Flower Hall_kVA
Ithemba Labs High Voltage->WITS Ithemba Labs High Voltage_kVA
Wits Science Stadium->WITS WC Science Staduim TRF 2_kVA
Mathematical Science Building->WITS WC Maths Science Building_kVA
Zesti Lemonz->WITS EC PDH Zesti Lemon_kVA
FNB Building->WITS WC FNB Building Total_kVA
PEC PIMD Office Block->WITS PEC PIMD Office Block_kVA
DJ Du Plesis->WITS WC DJ du Plessis Building_kVA
West Campus Village->WITS Campus Lodge_kVA
Convocation Dining Hall->WITS WC Convocation Dining Hall GEN 125 kVA_kVA
Barnato Residence->WITS WC Barnato Sub Residence A - D_kVA
David Webster Hall->WITS WC David Webster Hall_kVA
The Barns->WITS WC The Barns_kVA
Olives and Plates->WITS WBS Olives & Plates Meter 2_kVA
Johannesburg Planetarium->WITS EC Planetarium_kVA
Wartenweilier Library->WITS EC Wartenweiler Library_kVA
Origins Centre->WITS EC Origin Centre_kVA
Chemistry Building->WITS EC Physics Building_kVA
Physics Building->WITS E

## Preform Queries to load agrigated data into the mongoDB.

These queries MUST pull from the influxdb as this is where all the time series data is stored. As new records are added from Selenium (or directly from tnew sensors) they will be added to the Influx DB. this querying and updating mongo process occures every 30 minutes to generate "fresh" graphs.

First, we need to make the database store for each building

In [9]:
buildingInformationStore = db.buildingInformation

Then, itterate over all the buildings that have been identified in the geojson and query for values for that building from influxdb for the predefined length of time. For each element, influx db is queried and then the output is computed over to generate averages per day, week and year. The maximum building week is also extracted. This value is used later to generate the heat map.


In [10]:
startTime = "2018-08-19T00:00:00+00:00"

#Define a structure used in the creation of tables. itterate over each to define the graphs to draw
plotsToDraw = {
    'Day':{
        "resampleLength":'1H',
        "windowLength": 2*24, #1 day window with 2 samples per hour
        "moduloSize": 24
        },
    'Week':{
        "resampleLength":'1D',
        "windowLength":2*24*7, # 1 week window with 2 samples per hour
        "moduloSize": 7
        },
    'Year':{
        "resampleLength":'1M',
        "windowLength":2*24*30*8, #1 year window with 2 samples per hour
        "moduloSize": 8
        },
}

for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    #This blob is in accordance with the datastructure defined by swagger
    buildingBlob = {
        "BuildingId": index,
        "BuildingName": buildingName,
        "BensorName": "SENSOR",
        "ChartInformation": {
            "DayInformation": {
                "LastDay": [],
                "AverageDay": [],
                "LastDayAverage":0,
                "MaximumDay":0
            },
            "WeekInformation": {
                "LastWeek": [],
                "AverageWeek":[],
                "LastWeekAverage": 0,
                "MaximumWeek":0
            },
            "YearInformation": {          
                "LastYear": [],
                "AverageYear":[],
                "LastYearAverage":0,
                "MaximumYear":0
            }
        },
    }
    for plotType in plotsToDraw:
        #We want to get all the time series data for that particular building. The start time would be removed if live
        #data was added to the data set as we would want to query results from the current time back to the begining of the set
        query = "SELECT * FROM timeseriesdata WHERE buildingName='{}' AND time< '{}'".format(buildingName, startTime)
        queryResults = influxClient.query(query)
        #We need to extract the time series data from the influx query results
        results = queryResults["timeseriesdata"][queryResults["timeseriesdata"].keys()[0]]
        #For the resampled region(to calculculate the last day/week/year) we dont need the full set so sub
        #set using the tail (to get last x entries)
        resampledResults = results.tail(plotsToDraw[plotType]['windowLength']).resample(plotsToDraw[plotType]['resampleLength'], label='right').sum()
        #we can calculate the average overthe last period resampled region
        buildingBlob["ChartInformation"][plotType+"Information"]["Last" + plotType + "Average"]=resampledResults.mean()
        
        #Next, we want to convert the results into a format that mongo can accept. we cast it to a dict
        #and then itterate over all results
        resultsDict = resampledResults.to_dict()
        formattedDict = {} 
        for result in resultsDict:
            formattedDict[str(result)[0:19]] = resultsDict[result]
        buildingBlob["ChartInformation"][plotType+"Information"]["Last" + plotType] = formattedDict
        
        #The last step is to calculate the period average over all samples(eg the average week, over all data points)
        
        resampledResults_totalSet = results.resample(plotsToDraw[plotType]['resampleLength'], label='right').sum()

        buildingBlob["ChartInformation"][plotType+"Information"]["Maximum"+plotType]=np.max(resampledResults_totalSet)
        resampledResults_totalSetDict = resampledResults_totalSet.to_dict()
        
        timeSeriesValues =  [[] for _ in range(plotsToDraw[plotType]['moduloSize'])]
        for index, result in enumerate(resampledResults_totalSetDict):
            relativeIndex = index % plotsToDraw[plotType]['moduloSize']
            timeSeriesValues[relativeIndex].append(resampledResults_totalSetDict[result])
        
        finalizedAverage = []
        for result in timeSeriesValues:
            finalizedAverage.append(np.array(result).mean())
  
        #In order to plot these values correctly, they need to be bound off the last week key values        
        averageResultsDict = {}
        for index, key in enumerate(formattedDict):
            averageResultsDict[key] = finalizedAverage[index]
        buildingBlob["ChartInformation"][plotType+"Information"]["Average" + plotType]=averageResultsDict
    
#     print(buildingBlob)
    buildingInformation_id = buildingInformationStore.insert_one(buildingBlob).inserted_id
    print ("Building: {} Added with id {}".format(buildingName, buildingInformation_id))

Building: Chamber of Mines Added with id 5bc1afe1b2831026c7ca7819
Building: Flower Hall Added with id 5bc1afe4b2831026c7ca781a
Building: Ithemba Labs High Voltage Added with id 5bc1afe6b2831026c7ca781b
Building: Wits Science Stadium Added with id 5bc1afe8b2831026c7ca781c
Building: Mathematical Science Building Added with id 5bc1afeab2831026c7ca781d
Building: Zesti Lemonz Added with id 5bc1afebb2831026c7ca781e
Building: FNB Building Added with id 5bc1afecb2831026c7ca781f
Building: PEC PIMD Office Block Added with id 5bc1afedb2831026c7ca7820
Building: DJ Du Plesis Added with id 5bc1afefb2831026c7ca7821
Building: West Campus Village Added with id 5bc1aff0b2831026c7ca7822
Building: Convocation Dining Hall Added with id 5bc1aff1b2831026c7ca7823
Building: Barnato Residence Added with id 5bc1aff2b2831026c7ca7824
Building: David Webster Hall Added with id 5bc1aff3b2831026c7ca7825
Building: The Barns Added with id 5bc1aff4b2831026c7ca7826
Building: Olives and Plates Added with id 5bc1aff5b28310

# Calculate the university wide metrics
We want to be able to view the relative values for each set. These are averages over all time for all buildings. We already know what the average for each building is from before so we just need to find the average over all buildings for those particular sets.

In [11]:
buildingInformation = list(buildingInformationStore.find({}))

campusInformation = {
    "AveragePastDay": {},
    "AveragePastWeek": {},
    "AveragePastYear": {},
    "Maximums": {
        "Day": 0,
        "Week": 0,
        "Year": 0
    },
    "MaximumLastPeriodAverage": {
        "Day": 0,
        "Week": 0,
        "Year": 0
    }
}
#Itterate through all the plots that need averages
for plotType in plotsToDraw:
    typeAverageArray = []
    
    #For each building, extract their relevent time series values to sum over to get averages
    #Each position in the timeSeriesValues array corisponds to a day(it is an array of 7 for a week, for example)
    timeSeriesValues =  [[] for _ in range(plotsToDraw[plotType]['moduloSize'])]
    #itterate over all buildings
    for building in buildingInformation:
        spesificBuildingArray = building["ChartInformation"][plotType+"Information"]["Average" + plotType].values()
        for index, value in enumerate(spesificBuildingArray):
            timeSeriesValues[index].append(value)
    
    #calculate each everage and assign keys as time stamp values    
    for index, key in enumerate(building["ChartInformation"][plotType+"Information"]["Average" + plotType].keys()):
        numpyArray = np.array(timeSeriesValues[index])
        campusInformation["AveragePast"+plotType][key] = numpyArray.mean()
#         print(numpyArray.mean())
    
#         campusInformation["Maximums"][plotType] = np.max(numpyArray)
#         print(campusInformation["AveragePast"+plotType][key])
    
    #Find maximum and MaximumLastPeriodAverage for each time period and asign it to struct
    for building in buildingInformation:
        buildingMax = building["ChartInformation"][plotType+"Information"]["Maximum"+plotType]
        if buildingMax > campusInformation["Maximums"][plotType]:
            campusInformation["Maximums"][plotType] = buildingMax
        buildingAverage = building["ChartInformation"][plotType+"Information"]["Last"+plotType+"Average"]
        if buildingAverage > campusInformation["MaximumLastPeriodAverage"][plotType]:
            campusInformation["MaximumLastPeriodAverage"][plotType] = buildingAverage
print(campusInformation)
campusInfoStore = db.campusInfo
campusInfo_id = campusInfoStore.insert_one(campusInformation).inserted_id

KeyError: 'ChartInformation'

# Generate geojson for each location
Colours need to be assigned based off a buildings magnitude. 
First, define a set of functions to convert a max/min/value into a RGB value. This value is used to define the colour on the final heatmap

In [ ]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

def getHex(minimum,maximum,value):
    return '#%02x%02x%02x' % rgb(minimum,maximum,value)

In [ ]:
getHex(0,10,0)

Next, we itterate over the geojson set and update the square heat based of the set maximum

In [ ]:
 # We want to gives the colours for the heatmap to show the consumption averages & maximums over a day, week and year
groupings = ["Average","Maximum"]
for group in groupings:
    for plotType in plotsToDraw:
        for index, feature in enumerate(loadedGeoJson["features"]):
            buildingName = feature["properties"]["buildingName"]
            if group =="Average":
                value = buildingInformationStore.find_one({"BuildingName": buildingName})['ChartInformation'][plotType + 'Information']["Last" +plotType + group]
                maximum = campusInformation["MaximumLastPeriodAverage"][plotType]
            else:
                maximum = campusInformation["Maximums"][plotType]
                value = buildingInformationStore.find_one({"BuildingName": buildingName})['ChartInformation'][plotType + 'Information'][group + plotType]
            newColour = getHex(0,
                               maximum,
                               value)
            print(maximum,value,newColour)
#             print(buildingInformationStore.find_one({"BuildingName": buildingName})['ChartInformation'][plotType + 'Information']['Last'+ plotType + group])
            loadedGeoJson["features"][index]["properties"][plotType + "Style_" + group]["fillColor"] = newColour
            loadedGeoJson["features"][index]["properties"]["buildingId"] = index #set the index of the building based off the previous indecies
print(loadedGeoJson)
    

# Insert the geojson into the database
Now that the heatmap colours have been calculated, we can incert it into the database

In [ ]:
geojsonStore = db.geojson
geojson_id = geojsonStore.insert_one(loadedGeoJson).inserted_id

In [ ]:
loadedCampusInfo = {}
with open('../../Assets/otherinformation/campusInfomation.json') as json_data:
    loadedCampusInfo = json.load(json_data)
    print(loadedCampusInfo)

In [ ]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

In [ ]:
BuildingChartInformation = {}
with open('../../Assets/ChartInformation/BuildingChartInformation.json') as json_data:
    BuildingChartInformation = json.load(json_data)
    print(BuildingChartInformation)

In [ ]:
buildingInformationStore = db.buildingInformation
buildingInformation_id = buildingInformationStore.insert_one(BuildingChartInformation).inserted_id

In [ ]:
list(buildingInformationStore.find({"buildingId": "0"}))